## Week3 Assignment
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

More than one neighborhood can exist in one postal code area. 

If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

Submit a link to your Notebook on your Github repository.

## 1

In [1]:
#install 
!pip install beautifulsoup4
!pip install requests
!pip3 install lxml
!pip install lxml
!pip install geopy
!conda install -c conda-forge folium=0.5.0 --yes

     |████████████████████████████████| 115 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 6.3 MB 4.2 MB/s eta 0:00:01     |██████████████████████████████▋ | 6.0 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 118 kB 4.5 MB/s eta 0:00:01
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    ca-certificates-2021.5.30  

In [4]:
#import
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

from requests import get 
import lxml.html as lh
import json
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [8]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html.parser")  # create a soup object using the variable 'data'
table = soup.find('table')
table_rows = table.find_all('td')
table_rows

[<td style="width:11%;">
 <p>M1A<br/><span style="font-size:85%;">Not assigned</span>
 </p>
 </td>,
 <td style="width:11%;">
 <p>M2A<br/><span style="font-size:85%;">Not assigned</span>
 </p>
 </td>,
 <td style="width:11%;">
 <p>M3A<br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
 </p>
 </td>,
 <td style="width:11%;">
 <p>M4A<br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)</span>
 </p>
 </td>,
 <td style="width:11%;">
 <p>M5A<br/><span style="font-size:85%;"><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a><br/>(<a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a> / <a href="/wiki/Harbourfront,_Toronto" title="Harbourfront, Toronto">Harbourfront</a>)</span>
 </p>
 </td>,
 <td style="width:11%;">
 <p>M6A

In [14]:
# listing data - three columns: PostalCode, Borough, and Neighborhood 
postal_code = []
borough = []
neighbourhoods = []

for row in table_rows:
    string = row.text.replace('\n','')
    
    #Postal code
    post_code = string[0:3]
    postal_code.append(post_code)
    
    #Borough
    borough_data = string[3:].split('(')
    borough.append(borough_data[0])
    
    #Neighbourhood
    if len(borough_data) == 1:
        neighbourhoods.append(['Not Assigned'])
    elif len(borough_data)==2:
        neigh_clean = borough_data[1].replace(')','').split(' / ')
        neighbourhoods.append(neigh_clean)
    else:
        neighbourhoods.append(['Not Assigned'])
        
# Dataframe- Ignore cells with a borough that is Not assigned.
df = pd.DataFrame({'PostalCode':postal_code,'Borough':borough,'Neighbourhood':neighbourhoods})
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df = df.reset_index(drop=True)

df.loc[76,['Borough']] = 'Mississauga'
df.loc[92,['Borough']] = 'Downtown Toronto'
df.loc[100,['Borough']] = "East Toronto"

df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,"[Regent Park, Harbourfront]"
3,M6A,North York,"[Lawrence Manor, Lawrence Heights]"
4,M7A,Queen's Park,[Ontario Provincial Government]


In [15]:
# .shape method
df.shape

(103, 3)

## 2

Use the Geocoder package or the csv file to create the following dataframe:
(but Geocoder doesn't work)

In [30]:
# csv
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
data  = requests.get(url).text.split('\n')

# listing data of csv
post_codes = []
lati = []
longi = []
for d in data[1:]:
    split_string = d.split(',')
    post_codes.append(split_string[0])
    lati.append(split_string[1])
    longi.append(split_string[2])
    
# comparing
pc = []
b = []
n = []
la = []
lo = []

for p in range(len(postal_code)):
    for q in range(len(post_codes)):
        if postal_code[p] == post_codes[q]:
            pc.append(postal_code[p])
            b.append(borough[p])
            n.append(neighbourhoods[p])
            la.append(lati[q])
            lo.append(longi[q])
    
# create dataframe
df_geo = pd.DataFrame({'PostalCode':pc,'Borough':b,'Neighbourhood':n,"Latitude":la,"Longitude":lo})  

# replace
df_geo.loc[35,['Borough']] = 'East York'
df_geo.loc[76,['Borough']] = 'Mississauga'
df_geo.loc[92,['Borough']] = 'Downtown Toronto'
df_geo.loc[94,['Borough']] = 'Etobicoke'
df_geo.loc[100,['Borough']] = "East Toronto"

df_geo.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,[Parkwoods],43.7532586,-79.3296565
1,M4A,North York,[Victoria Village],43.7258823,-79.3155716
2,M5A,Downtown Toronto,"[Regent Park, Harbourfront]",43.6542599,-79.3606359
3,M6A,North York,"[Lawrence Manor, Lawrence Heights]",43.718518,-79.4647633
4,M7A,Queen's Park,[Ontario Provincial Government],43.6623015,-79.3894938


## 3

Explore and cluster the neighborhoods in Toronto. 
You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. 
Just make sure:
to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
to generate maps to visualize your neighborhoods and how they cluster together. 

In [35]:
address='Toronto, TO'
geolocater=Nominatim(user_agent="toronto_explorer")
location = geolocater.geocode(address)                     
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.65238435, -79.38356765.
